# Assignment 1

![Assignment 1](assets/assn1.png)
![Assignment 2](assets/assn2.png)

## Functions Used

### Gaussian function

![Gaussian](assets/gaussian.png)

### Gaussian x derivative

![Gaussian X](assets/gauss_der_x.png)

### Gaussian y derivative

![Gaussian Y](assets/gauss_der_y.png)


In [1]:
import numpy as np
import cv2

In [2]:
img = cv2.imread('../../../assets/buildings.jpg', cv2.IMREAD_GRAYSCALE)
cv2.imshow('Original Image', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Gaussian functions

In [3]:
def gaussian(u, v, sigma = 1):
    g = np.exp(-(u**2 + v**2) / (2 * sigma**2)) / (2 * np.pi * sigma**2)
    return g

def gaussian_derivative_x(u, v, sigma):
    g_x = -u * gaussian(u, v, sigma) / (sigma**2)
    return g_x

def gaussian_derivative_y(u, v, sigma):
    g_y = -v * gaussian(u, v, sigma) / (sigma**2)
    return g_y

## Generate 7x7 Gaussian x and y derivative kernels

In [4]:
def gaussian_derivative_kernels(m=7, sigma=1):
    assert m % 2 == 1, "Kernel size must be odd"
    k = (m - 1) // 2
    kernel_x = np.zeros((m, m), dtype=np.float32)
    kernel_y = np.zeros((m, m), dtype=np.float32)
    for i in range(m):
        for j in range(m):
            kernel_x[i, j] = gaussian_derivative_x(i - k, j - k, sigma)
            kernel_y[i, j] = gaussian_derivative_y(i - k, j - k, sigma)
    return kernel_x, kernel_y

In [5]:
kernel_x, kernel_y = gaussian_derivative_kernels()

print('='*60)
print("KERNEL (X direction):")
print('='*60)
print(kernel_x)

print('='*60)
print("\nKERNEL (Y direction):")
print('='*60)
print(kernel_y)


KERNEL (X direction):
[[ 5.8923841e-05  7.1783934e-04  3.2171328e-03  5.3041549e-03
   3.2171328e-03  7.1783934e-04  5.8923841e-05]
 [ 4.7855955e-04  5.8300490e-03  2.6128467e-02  4.3078560e-02
   2.6128467e-02  5.8300490e-03  4.7855955e-04]
 [ 1.0723775e-03  1.3064234e-02  5.8549833e-02  9.6532352e-02
   5.8549833e-02  1.3064234e-02  1.0723775e-03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.0723775e-03 -1.3064234e-02 -5.8549833e-02 -9.6532352e-02
  -5.8549833e-02 -1.3064234e-02 -1.0723775e-03]
 [-4.7855955e-04 -5.8300490e-03 -2.6128467e-02 -4.3078560e-02
  -2.6128467e-02 -5.8300490e-03 -4.7855955e-04]
 [-5.8923841e-05 -7.1783934e-04 -3.2171328e-03 -5.3041549e-03
  -3.2171328e-03 -7.1783934e-04 -5.8923841e-05]]

KERNEL (Y direction):
[[ 5.8923841e-05  4.7855955e-04  1.0723775e-03  0.0000000e+00
  -1.0723775e-03 -4.7855955e-04 -5.8923841e-05]
 [ 7.1783934e-04  5.8300490e-03  1.3064234e-02  0.0000000e+00
  -1.3064234

## Apply filtering in x and y direction

In [6]:
img_dx = cv2.filter2D(img, cv2.CV_64F, kernel_x)
img_dy = cv2.filter2D(img, cv2.CV_64F, kernel_y)

img_dx_norm = np.round(cv2.normalize(img_dx, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)).astype(np.uint8)
img_dy_norm = np.round(cv2.normalize(img_dy, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)).astype(np.uint8)

cv2.imshow('Gradient X', img_dx_norm)
cv2.imshow('Gradient Y', img_dy_norm)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Compute gradient magnitude

In [7]:
img_grad = cv2.magnitude(img_dx.astype(np.float32), img_dy.astype(np.float32))
img_grad_norm = np.round(cv2.normalize(img_grad, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)).astype(np.uint8)

cv2.imshow('Gradient Magnitude', img_grad_norm)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Apply double thresholding

In [8]:
def double_threshold(image, T_min=64, T_max=128):
    res = np.zeros_like(image, dtype=np.uint8)
    weak = np.zeros_like(image, dtype=np.uint8)
    strong = np.zeros_like(image, dtype=np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i, j] < T_min:
                res[i, j] = 0
            elif image[i, j] < T_max:
                weak[i, j] = 128
                res[i, j] = 128
            else:
                strong[i, j] = 255
                res[i, j] = 255
    return res, weak, strong

In [9]:
img_double, img_weak, img_strong = double_threshold(img_grad_norm)

cv2.imshow('Weak Edges', img_weak)
cv2.imshow('Strong Edges', img_strong)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Apply hysteresis thresholding

In [10]:
def hysteresis_threshold(image, T_min=25, T_max=100):
    res, _, _ = double_threshold(image, T_min, T_max)

    strong_coords = np.argwhere(res == 255)
    h, w = res.shape

    neighbors = [(-1,-1), (-1,0), (-1,1),
                (0,-1),         (0,1),
                (1,-1), (1,0), (1,1)]

    while len(strong_coords) > 0:
        y, x = strong_coords[-1]
        strong_coords = strong_coords[:-1]

        for dy, dx in neighbors:
            ny, nx = y + dy, x + dx
            if 0 <= ny < h and 0 <= nx < w:
                if res[ny, nx] == 128:
                    res[ny, nx] = 255
                    strong_coords = np.vstack([strong_coords, [ny, nx]])

    res[res != 255] = 0
    return res


In [11]:
img_hyst = hysteresis_threshold(img_grad_norm, 64, 128)

In [12]:
cv2.imshow("Original Image", img)
cv2.imshow("Double Threshold", img_double)
cv2.imshow("Hysteresis Threshold", img_hyst)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Apply Canny Edge Detection using built-in functions

In [13]:
img_blur = cv2.GaussianBlur(img, (5, 5), 0)
img_canny = cv2.Canny(img_blur, 64, 128)

## Compare the results

In [14]:
cv2.imshow("Original Image", img)
cv2.imshow("Double Thresholding", img_double)
cv2.imshow('Hysteresis Thresholding', img_hyst)
cv2.imshow('Canny Edge Detection', img_canny)

cv2.waitKey(0)
cv2.destroyAllWindows()